## Simulate solving the inverse step with Conjugate Gradient (CG)

In [1]:
import numpy as np
import scipy
from scipy.sparse.linalg import cg, spilu, LinearOperator
import time

In [2]:
N = 1000
d_avg = 3
sigma = 0.1

In [3]:
# 1. Generate a random sparse kernel matrix K, with O(N) non-zero entries.

Phi = scipy.sparse.random(N,N,d_avg/N, format='dense')

K = Phi @ Phi.T # K is SPD

I = np.eye(N)

M = K + sigma**2 * I

# print the percentage of non-zero entries in the kernel matrix
print(f"Percentage of non-zero entries in K: {np.count_nonzero(K) / (N * N) * 100:.2f}%")

Percentage of non-zero entries in K: 1.00%


In [4]:
# 2. Baseline: Direct Inversion


# Numpy direct inversion
start_time = time.time()
M_inv_1 = np.linalg.inv(M)
end_time = time.time()
print(f"Numpy direct inversion took {end_time - start_time:.4f} seconds")


# Scipy direct inversion
start_time = time.time()
M_inv_2= scipy.linalg.inv(M)
end_time = time.time()
print(f"Scipy direct inversion took {end_time - start_time:.4f} seconds")

# Inversion with scipy sparse linear algebra
M_csc = scipy.sparse.csc_matrix(M)

start_time = time.time()
M_inv_3 = scipy.sparse.linalg.inv(M_csc)
end_time = time.time()

print(f"Scipy sparse linear algebra inversion took {end_time - start_time:.4f} seconds")


Numpy direct inversion took 0.0447 seconds
Scipy direct inversion took 0.0284 seconds
Scipy sparse linear algebra inversion took 0.3718 seconds


In [5]:
# 3. Conjugate Gradient (CG) Method - note CG requires SPD matrices
# Solve M @ x = b, where b is a dense vector
b  = np.random.rand(N)

start_time = time.time()
x, info = cg(M_csc, b, atol=1e-10, maxiter=1000)
end_time = time.time()

print(f"Preconditioned CG took {end_time - start_time:.4f} seconds")
if info == 0:
    print("Converged to tolerance")
elif info > 0:
    print(f"Stopped after {info} iterations without full convergence")
else:
    print("Illegal input or breakdown in CG")

Preconditioned CG took 0.0045 seconds
Converged to tolerance


In [6]:
# 4. Preconditioned Conjugate Gradient (PCG) Method

# Build your sparse SPD matrix in CSC (for spilu) and CSR (for mat-vec)
M_csc = scipy.sparse.csc_matrix(M)
M_csr = M_csc.tocsr()

# Right-hand side
b = np.random.rand(N)

# Tuned ILU parameters:
#  - drop_tol: lower drops less (better approximation), but more fill
#  - fill_factor: allows more fill-in beyond original sparsity
#  - permc_spec: use a fill-reducing ordering (COLAMD)
#  - diag_pivot_thresh: avoid tiny pivots
ilu = spilu(
    M_csc,
    drop_tol=1e-6,           # tighter drop tolerance
    fill_factor=20,          # allow more fill-in
    permc_spec="COLAMD",     # fill-reducing column ordering
    diag_pivot_thresh=0.1    # pivot threshold to maintain stability
)

# Wrap as a LinearOperator for CG
M_prec = LinearOperator(
    M_csr.shape,
    matvec=lambda x: ilu.solve(x)
)

# Run preconditioned CG
start_time = time.time()
x, info = cg(
    M_csr,
    b,
    M=M_prec,
    atol=1e-10,
    maxiter=10000
)
end_time = time.time()

print(f"Preconditioned CG took {end_time - start_time:.4f} seconds")
if info == 0:
    print("Converged to tolerance")
elif info > 0:
    print(f"Stopped after {info} iterations without full convergence")
else:
    print("Illegal input or breakdown in CG")


Preconditioned CG took 1.2476 seconds
Stopped after 10000 iterations without full convergence
